In [1]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

import spacy
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ravit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ravit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import pandas as pd
import os
data=pd.read_csv("20190928-reviews.csv")
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [19]:
data = data.drop(columns=['asin', 'name', 'date','verified','helpfulVotes'], axis=1)
data.head()

,rating,title,body
0,3,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,5,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,3,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


In [4]:
data = data.dropna(axis=0, subset=['body'])

In [5]:
data['body'].isnull().sum()

0

In [6]:
data = data.body.values.tolist()

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['had', 'the', 'samsung', 'for', 'awhile', 'which', 'is', 'absolute', 'doo', 'doo', 'you', 'can', 'read', 'my', 'review', 'on', 'it', 'and', 'detect', 'my', 'rage', 'at', 'the', 'stupid', 'thing', 'it', 'finally', 'died', 'on', 'me', 'so', 'used', 'this', 'nokia', 'phone', 'bought', 'in', 'garage', 'sale', 'for', 'wonder', 'she', 'sold', 'it', 'so', 'cheap', 'bad', 'hate', 'the', 'menu', 'it', 'takes', 'forever', 'to', 'get', 'to', 'what', 'you', 'want', 'because', 'you', 'have', 'to', 'scroll', 'endlessly', 'usually', 'phones', 'have', 'numbered', 'categories', 'so', 'can', 'simply', 'press', 'the', 'and', 'get', 'where', 'you', 'want', 'to', 'go', 'it', 'pain', 'to', 'put', 'it', 'on', 'silent', 'or', 'vibrate', 'if', 'you', 're', 'in', 'class', 'and', 'it', 'rings', 'you', 'have', 'to', 'turn', 'it', 'off', 'immediately', 'there', 'no', 'fast', 'way', 'to', 'silence', 'the', 'damn', 'thing', 'always', 'remember', 'to', 'put', 'it', 'on', 'silent', 'learned', 'that', 'the', 'hard', 

In [8]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['had', 'the', 'samsung', 'for', 'awhile', 'which', 'is', 'absolute', 'doo', 'doo', 'you', 'can', 'read', 'my', 'review', 'on', 'it', 'and', 'detect', 'my', 'rage', 'at', 'the', 'stupid', 'thing', 'it', 'finally', 'died', 'on', 'me', 'so', 'used', 'this', 'nokia', 'phone', 'bought', 'in', 'garage', 'sale', 'for', 'wonder', 'she', 'sold', 'it', 'so', 'cheap', 'bad', 'hate', 'the', 'menu', 'it', 'takes', 'forever', 'to', 'get', 'to', 'what', 'you', 'want', 'because', 'you', 'have', 'to', 'scroll', 'endlessly', 'usually', 'phones', 'have', 'numbered', 'categories', 'so', 'can', 'simply', 'press', 'the', 'and', 'get', 'where', 'you', 'want', 'to', 'go', 'it', 'pain', 'to', 'put', 'it', 'on', 'silent', 'or', 'vibrate', 'if', 'you', 're', 'in', 'class', 'and', 'it', 'rings', 'you', 'have', 'to', 'turn', 'it', 'off', 'immediately', 'there', 'no', 'fast', 'way', 'to', 'silence', 'the', 'damn', 'thing', 'always', 'remember', 'to', 'put', 'it', 'on', 'silent', 'learned', 'that', 'the', 'hard', '

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
import en_core_web_sm
#nlp = spacy.load('en')


In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = en_core_web_sm.load()
#nlp = spacy.load('en',parse=True,tag=True, entity=True)
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['awhile', 'absolute', 'read', 'review', 'detect', 'rage', 'stupid', 'thing', 'finally', 'die', 'use', 'phone', 'buy', 'garage', 'sale', 'wonder', 'sell', 'cheap', 'bad', 'hate', 'menu', 'take', 'forever', 'get', 'want', 'scroll', 'endlessly', 'usually', 'phone', 'number', 'category', 'simply', 'press', 'get', 'want', 'go', 'pain', 'put', 'silent', 'vibrate', 'class', 'ring', 'turn', 'immediately', 'fast', 'way', 'silence', 'damn', 'thing', 'always', 'remember', 'put', 'silent', 'learn', 'hard', 'way', 'true', 'case', 'mission', 'get', 'break', 'nail', 'process', 'also', 'damage', 'case', 'time', 'try', 'reason', 'phone', 'start', 'give', 'problem', 'succeed', 'opening', 'button', 'bit', 'big', 'vibration', 'stronger', 'good', 'reception', 'use', 'elevator', 'remarkable', 'feat', 'consider', 'old', 'phone', 'lose', 'service', 'simply', 'put', 'pocket', 'compare', 'old', 'samsung', 'phone', 'work', 'quite', 'well', 'ring_tone', 'loud', 'enough', 'hear', 'phone', 'actually', 'charge', '

In [12]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 4), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 9), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 3), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 2), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110, 1)

In [13]:
id2word[0]

'absolute'

In [14]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolute', 1),
  ('actually', 1),
  ('also', 1),
  ('always', 1),
  ('aqua', 1),
  ('awhile', 1),
  ('bad', 1),
  ('bar', 1),
  ('battery', 1),
  ('big', 1),
  ('bit', 1),
  ('break', 1),
  ('bright', 1),
  ('button', 1),
  ('buy', 1),
  ('case', 2),
  ('category', 1),
  ('charge', 1),
  ('cheap', 1),
  ('class', 1),
  ('come', 1),
  ('compare', 1),
  ('comparison', 1),
  ('consider', 1),
  ('convos', 1),
  ('customize', 1),
  ('cute', 1),
  ('damage', 1),
  ('damn', 1),
  ('definitely', 1),
  ('detect', 1),
  ('die', 1),
  ('elevator', 1),
  ('endlessly', 1),
  ('enough', 1),
  ('fast', 1),
  ('feat', 1),
  ('finally', 1),
  ('forever', 1),
  ('garage', 1),
  ('get', 4),
  ('give', 1),
  ('go', 1),
  ('good', 1),
  ('great', 2),
  ('hard', 1),
  ('hate', 1),
  ('hear', 1),
  ('heat', 1),
  ('immediately', 1),
  ('large', 1),
  ('learn', 1),
  ('life', 1),
  ('long', 1),
  ('lose', 1),
  ('loud', 1),
  ('menu', 1),
  ('mission', 1),
  ('nail', 1),
  ('new', 1),
  ('nice', 1),
  ('n

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.171*"phone" + 0.034*"work" + 0.027*"get" + 0.020*"buy" + 0.017*"come" + 0.016*"go" + 0.014*"new" + 0.013*"charge" + 0.013*"day" + 0.011*"time"'), (1, '0.075*"app" + 0.048*"device" + 0.037*"support" + 0.028*"software" + 0.027*"storage" + 0.027*"option" + 0.024*"datum" + 0.017*"usb" + 0.016*"compatible" + 0.013*"available"'), (2, '0.040*"good" + 0.036*"great" + 0.029*"screen" + 0.028*"camera" + 0.023*"battery" + 0.020*"price" + 0.017*"love" + 0.016*"well" + 0.015*"really" + 0.014*"fast"'), (3, '0.060*"product" + 0.047*"awesome" + 0.045*"performance" + 0.039*"hand" + 0.038*"flagship" + 0.038*"moto" + 0.034*"include" + 0.029*"quick" + 0.018*"market" + 0.014*"item"'), (4, '0.041*"call" + 0.026*"try" + 0.021*"return" + 0.020*"receive" + 0.018*"unlock" + 0.018*"turn" + 0.017*"service" + 0.016*"make" + 0.015*"set" + 0.015*"network"')]


## Coherence of the model

In [17]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4292921598599338


In [1]:
import pandas as pd
import numpy as np
import re
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from gensim.models import ldamodel
#import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
from sklearn import datasets
from sklearn.preprocessing import normalize
import pickle

In [2]:
import pandas as pd
import os
data=pd.read_csv("20190928-reviews.csv")
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [3]:
data = data.drop(columns=['asin', 'name', 'date','verified','helpfulVotes'], axis=1)
data.head()

,rating,title,body
0,3,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,5,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,3,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


In [4]:
data = data.dropna(axis=0, subset=['body'])

In [5]:
def top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

## LSA

In [9]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(data.body)
nmf = NMF(n_components=5, random_state=1,
          alpha=.1, l1_ratio=.5)
nmf.fit(data_tf_idf)
nmf_data=nmf.transform(data_tf_idf)

In [10]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: phone battery screen use work just nice camera life best
Topic #1: good product price far quality condition really phone battery value
Topic #2: great phone price works product condition fast value deal far
Topic #3: love phone absolutely new thank note far amazing awesome fast
Topic #4: works new like brand perfect looks condition came excellent just



## PLSA

In [20]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(data.body)
nmf = NMF(n_components=5, random_state=1, beta_loss='kullback-leibler',
          solver='mu', alpha=.1, l1_ratio=.5)
nmf_data=nmf.fit(data_tf_idf)

In [21]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: phone screen use work battery just time camera phones samsung
Topic #1: good battery far quality life really price bad pretty value
Topic #2: great works product price value condition far shipping deal shape
Topic #3: love thank great absolutely price far thanks color learning value
Topic #4: new works like perfect excellent came looks condition brand excelente



## LDA

In [22]:
count = CountVectorizer(stop_words='english', max_features=1000)
data_count = count.fit_transform(data.body)
lda = LatentDirichletAllocation(n_components=5, max_iter=5, 
                                learning_method='online', random_state=1)
lda_data=lda.fit_transform(data_count)

In [23]:
top_words(lda, count.get_feature_names(), 10)

Topic #0: phone new like came love charger brand perfect refurbished box
Topic #1: android product google issue device el software version samsung nokia
Topic #2: phone screen camera battery like use quality phones just life
Topic #3: phone work sim card unlocked did verizon just use mobile
Topic #4: great phone good works price far battery fast happy condition

